In [2]:
import numpy as np
import pandas as pd
import warnings
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support




# Internal Data
 

In [3]:
Internal_path = Path.cwd().parent / "data"/ "interim" / "case_study1.xlsx"


# Cibil External data

In [4]:
External_path = Path.cwd().parent / "data"/  "external" / "case_study2.xlsx"

In [5]:
df1 = pd.read_excel(External_path)
df2 = pd.read_excel(Internal_path)

 # Internal Data clean up
 

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

In [16]:
def null_value(df):
    columns_remove_list = []
    for i in df.columns:
        if df.loc[df[i] == -99999].shape[0] > 10000:
            columns_remove_list.append(i)
    return columns_remove_list

In [18]:
null_value(df1)

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [69]:
def clean_data(df):
    return (
        df
        .drop(null_value(df),axis=1)
        .replace(-99999, np.nan)  # Replace -99999 with NaN
        .dropna()
    )#.isna().sum()
   

In [70]:
clean_data(df1)

,PROSPECTID,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549.0,11,29,0,0,0,0,0,21,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,47.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302.0,9,25,1,9,8,0,0,10,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
4,5,583.0,0,0,0,0,0,0,0,53,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
5,6,245.0,14,270,0,0,0,13,11,5,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15.0,2,24,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57.0,0,0,0,0,0,0,0,6,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
51333,51334,32.0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
51334,51335,58.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [72]:
clean_data(df2).columns

Index(['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL',
       'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M',
       'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl',
       'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL',
       'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL'],
      dtype='object')

In [73]:
for i in clean_data(df1).columns:
    if i in  clean_data(df2).columns:
        print(i)
    

PROSPECTID


In [74]:
df = pd.merge(clean_data(df1),clean_data(df2),how='inner',left_on= ["PROSPECTID"],right_on=['PROSPECTID'])

In [75]:
df

,PROSPECTID,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,549.0,11,29,0,0,0,0,0,21,...,0,0,1,0,4,1,4,0,72.0,18.0
1,2,47.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,7.0,7.0
2,3,302.0,9,25,1,9,8,0,0,10,...,0,6,1,0,0,2,6,0,47.0,2.0
3,5,583.0,0,0,0,0,0,0,0,53,...,0,0,0,0,0,3,0,2,131.0,32.0
4,6,245.0,14,270,0,0,0,13,11,5,...,0,0,2,0,0,6,0,0,150.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,15.0,2,24,0,0,0,0,0,0,...,0,2,0,0,0,0,3,1,24.0,5.0
42060,51333,57.0,0,0,0,0,0,0,0,6,...,0,2,0,0,0,2,2,0,74.0,7.0
42061,51334,32.0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,2,0,9.0,5.0
42062,51335,58.0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,2,0,15.0,8.0


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   time_since_recent_payment   42064 non-null  float64
 2   num_times_delinquent        42064 non-null  int64  
 3   max_recent_level_of_deliq   42064 non-null  int64  
 4   num_deliq_6mts              42064 non-null  int64  
 5   num_deliq_12mts             42064 non-null  int64  
 6   num_deliq_6_12mts           42064 non-null  int64  
 7   num_times_30p_dpd           42064 non-null  int64  
 8   num_times_60p_dpd           42064 non-null  int64  
 9   num_std                     42064 non-null  int64  
 10  num_std_6mts                42064 non-null  int64  
 11  num_std_12mts               42064 non-null  int64  
 12  num_sub                     42064 non-null  int64  
 13  num_sub_6mts                420